In [6]:
import pyodbc 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

### 1. Reading input data

In [2]:
import sqlalchemy as sa
from sqlalchemy import create_engine, event
from sqlalchemy.engine.url import URL

## Reading WILDFIRE data at 10km:
print ("connect to engine......")
engine = sa.create_engine('mssql+pyodbc://' + "GREENMONKEY" + '/' + "Climate_Impact" + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')

print ("connection to greenmonkey found")
print ("Reading wilfdire table from greenmonkey")
query=('''
SELECT     
      [GridNum10km]   
      ,[fire_date]
      ,sum([burned_area]) as [burned_area]
  FROM [Climate_Impact].[wild_fire].[wild_fires_2000_2022_light]
      
group by 
    [GridNum10km]
    ,[fire_date]
''')

df_wildfire10km = pd.read_sql(query, engine)
print(df_wildfire10km.head(26))

connect to engine......
connection to greenmonkey found
Reading wilfdire table from greenmonkey
     GridNum10km  fire_date  burned_area
0   2.317393e+16 2004-08-01           87
1   2.774463e+16 2020-10-01            2
2   2.769272e+16 2015-09-01           31
3   1.411224e+16 2021-04-01           68
4   9.695927e+15 2004-01-01          200
5   2.308242e+16 2021-02-01            6
6   1.870520e+16 2021-03-01           41
7   2.774550e+16 2016-07-01          367
8   9.729042e+15 2016-08-01          834
9   3.210836e+16 2015-08-01          124
10  2.311725e+16 2015-11-01         1790
11  2.765513e+16 2020-11-01            8
12  2.310014e+16 2015-09-01          752
13  9.713167e+15 2020-09-01           14
14  2.282758e+16 2004-07-01          278
15  9.713094e+15 2015-08-01           95
16  2.291548e+16 2020-11-01          144
17  9.713236e+15 2021-05-01           13
18  2.308363e+16 2021-03-01           73
19  1.894922e+16 2004-08-01           30
20  2.293157e+16 2016-07-01           32
21

In [3]:
## Reading AGB and transforming the df into a new schema:
print ("connect to engine......")
engine = sa.create_engine('mssql+pyodbc://' + "GREENMONKEY" + '/' + "Climate_Impact" + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')

print ("connection to greenmonkey found")
print ("Reading wilfdire table from greenmonkey")
query3=('''
        SELECT *  
     from   
       [Climate_Impact].wild_fire.[F_AGBflux_sum]
        ''')  
df_F_AGBflux_sum = pd.read_sql(query3, engine)
print(df_F_AGBflux_sum.head(26))

connect to engine......
connection to greenmonkey found
Reading wilfdire table from greenmonkey
          GridNum10km   01-11-2014   01-11-2015   01-11-2016   01-11-2017  \
0   22821536360562688          0.0          0.0          0.0          0.0   
1    9860785350115328          0.0          0.0          0.0          0.0   
2   27660980955643904          0.0          0.0          0.0          0.0   
3   18883523796533248          0.0          0.0          0.0          0.0   
4   14521696744636416          0.0          0.0          0.0          0.0   
5    4842545561468928          0.0          0.0          0.0          0.0   
6    5207660731301888          0.0          0.0          0.0          0.0   
7   14657941697200128          0.0          0.0          0.0          0.0   
8    5144713690611712          0.0          0.0          0.0          0.0   
9    9958586050412544          0.0          0.0          0.0          0.0   
10  13814440185036800          0.0          0.0          

In [5]:
df_transformed_AGBflux =df_F_AGBflux_sum.melt(id_vars=["GridNum10km","AreaHa"], 
     var_name="year", 
        value_name="GDMP")
## update date format:
df_transformed_AGBflux['year'] = pd.to_datetime(df_transformed_AGBflux['year'], format='%d-%m-%Y')
print(df_transformed_AGBflux.head(20))

          GridNum10km   AreaHa       year         GDMP
0   22821536360562688  10000.0 2014-11-01          0.0
1    9860785350115328  10000.0 2014-11-01          0.0
2   27660980955643904  10000.0 2014-11-01          0.0
3   18883523796533248   9429.0 2014-11-01          0.0
4   14521696744636416  10000.0 2014-11-01          0.0
5    4842545561468928  10000.0 2014-11-01          0.0
6    5207660731301888  10000.0 2014-11-01          0.0
7   14657941697200128  10000.0 2014-11-01          0.0
8    5144713690611712  10000.0 2014-11-01          0.0
9    9958586050412544  10000.0 2014-11-01          0.0
10  13814440185036800  10000.0 2014-11-01          0.0
11  10176495511142400  10000.0 2014-11-01          0.0
12  27731023282307072  10000.0 2014-11-01          0.0
13   5507681376796672  10000.0 2014-11-01          0.0
14   6021290745921536  10000.0 2014-11-01          0.0
15  27977696139018240  10000.0 2014-11-01   97412981.0
16  14185147402289152  10000.0 2014-11-01  762397386.0
17  189657

### 2. Computing variables

#### 2.1. GDMP anomalies

In [64]:
def z_score(x):
   z = np.around(zscore(x),1)
   return z

#df['Z_SCORE'] = df.groupby('GROUP')['VALUE'].transform(z_score)
df_transformed_AGBflux['zscore'] = df_transformed_AGBflux.groupby('GridNum10km')['GDMP'].transform(z_score)

In [74]:
GDMP = df_transformed_AGBflux[df_transformed_AGBflux['GDMP']>0]
GDMP.head()

,GridNum10km,AreaHa,year,GDMP,zscore
15,27977696139018240,10000.0,2014-11-01,97412981.0,0.8
16,14185147402289152,10000.0,2014-11-01,762397386.0,1.4
17,18965789600120832,10000.0,2014-11-01,588245574.0,0.3
18,27603260890152960,10000.0,2014-11-01,290296014.0,-1.0
19,9731949216137216,10000.0,2014-11-01,757097776.0,0.3


In [75]:
# create a list of our conditions
conditions = [
    (GDMP['zscore'] <= -1),
    (GDMP['zscore'] > -1) & (GDMP['zscore'] <= 1),
    (GDMP['zscore'] > 1)
    ]
     
# create a list of the values we want to assign for each condition
values = ['-1', '0', '+1']

# create a new column and use np.select to assign values to it using our lists as arguments
GDMP['ANOMALIES'] = np.select(conditions, values)

# display updated DataFrame
GDMP.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,GridNum10km,AreaHa,year,GDMP,zscore,ANOMALIES
15,27977696139018240,10000.0,2014-11-01,97412981.0,0.8,0
16,14185147402289152,10000.0,2014-11-01,762397386.0,1.4,+1
17,18965789600120832,10000.0,2014-11-01,588245574.0,0.3,0
18,27603260890152960,10000.0,2014-11-01,290296014.0,-1.0,-1
19,9731949216137216,10000.0,2014-11-01,757097776.0,0.3,0


##### Export table

In [81]:
name_of_table ="GDMP_Anomalies_10k"
     
print ("connect to greenmonkey engine.....GREENMONKEY.")  
engine = sa.create_engine('mssql+pyodbc://' + "GREENMONKEY" + '/' + "Climate_Impact" + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')
#print ("send table togreenmonkey MS-SQL:")
GDMP.to_sql(name_of_table, engine, if_exists='replace', index = False, schema='wild_fire')


connect to greenmonkey engine.....GREENMONKEY.


ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 13 for SQL Server][SQL Server]CREATE TABLE permission denied in database 'Climate_Impact'. (262) (SQLExecDirectW)")
[SQL: 
CREATE TABLE wild_fire.[GDMP_Anomalies_10k] (
	[GridNum10km] BIGINT NULL, 
	[AreaHa] FLOAT(53) NULL, 
	year DATETIME NULL, 
	[GDMP] FLOAT(53) NULL, 
	zscore FLOAT(53) NULL, 
	[ANOMALIES] VARCHAR(max) NULL
)

]
(Background on this error at: http://sqlalche.me/e/13/f405)

##### QC GDMP anomalies

In [65]:
print (df_transformed_AGBflux.head())

         GridNum10km   AreaHa       year  GDMP  zscore
0  22821536360562688  10000.0 2014-11-01   0.0     NaN
1   9860785350115328  10000.0 2014-11-01   0.0     NaN
2  27660980955643904  10000.0 2014-11-01   0.0     NaN
3  18883523796533248   9429.0 2014-11-01   0.0     NaN
4  14521696744636416  10000.0 2014-11-01   0.0     NaN


In [72]:
df_transformed_AGBflux[df_transformed_AGBflux['GridNum10km']== 22821536360562688]

,GridNum10km,AreaHa,year,GDMP,zscore
0,22821536360562688,10000.0,2014-11-01,0.0,NaN
302889,22821536360562688,10000.0,2015-11-01,0.0,NaN
605778,22821536360562688,10000.0,2016-11-01,0.0,NaN
908667,22821536360562688,10000.0,2017-11-01,0.0,NaN
1211556,22821536360562688,10000.0,2018-11-01,0.0,NaN
1514445,22821536360562688,10000.0,2019-11-01,0.0,NaN
1817334,22821536360562688,10000.0,2020-11-01,0.0,NaN
2120223,22821536360562688,10000.0,2021-11-01,0.0,NaN


In [66]:
df_transformed_AGBflux[df_transformed_AGBflux['GridNum10km']== 18446609658413056]

,GridNum10km,AreaHa,year,GDMP,zscore
114716,18446609658413056,10000.0,2014-11-01,567420786.0,-0.7
417605,18446609658413056,10000.0,2015-11-01,554946035.0,-1.1
720494,18446609658413056,10000.0,2016-11-01,591739566.0,0.2
1023383,18446609658413056,10000.0,2017-11-01,556189307.0,-1.1
1326272,18446609658413056,10000.0,2018-11-01,626023208.0,1.4
1629161,18446609658413056,10000.0,2019-11-01,597534185.0,0.4
1932050,18446609658413056,10000.0,2020-11-01,634120631.0,1.6
2234939,18446609658413056,10000.0,2021-11-01,565236683.0,-0.7


In [47]:
GDMP_grid1=df_transformed_AGBflux[df_transformed_AGBflux['GridNum10km']== 18446609658413056]
avg = GDMP_grid1['GDMP'].mean()
std = GDMP_grid1['GDMP'].std()
#GDMP_grid2['zscore'] = round(((GDMP_grid2['GDMP']-avg)/std),1)
GDMP_grid1.loc[:,'zscore'] = round(((GDMP_grid1['GDMP']-avg)/std),1)
GDMP_grid1

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,GridNum10km,AreaHa,year,GDMP,zscore
114716,18446609658413056,10000.0,2014-11-01,567420786.0,-0.6
417605,18446609658413056,10000.0,2015-11-01,554946035.0,-1.0
720494,18446609658413056,10000.0,2016-11-01,591739566.0,0.2
1023383,18446609658413056,10000.0,2017-11-01,556189307.0,-1.0
1326272,18446609658413056,10000.0,2018-11-01,626023208.0,1.3
1629161,18446609658413056,10000.0,2019-11-01,597534185.0,0.4
1932050,18446609658413056,10000.0,2020-11-01,634120631.0,1.5
2234939,18446609658413056,10000.0,2021-11-01,565236683.0,-0.7


In [16]:
zsc= stats.zscore(GDMP_grid['GDMP'])
zsc

array([-0.66404131, -1.09480203,  0.17570091, -1.05187109,  1.35953585,
        0.37579261,  1.63914477, -0.73945971])

In [45]:
GDMP_grid2=df_transformed_AGBflux[df_transformed_AGBflux['GridNum10km']== 9731949216137216]
avg = GDMP_grid2['GDMP'].mean()
std = GDMP_grid2['GDMP'].std()
#GDMP_grid2['zscore'] = round(((GDMP_grid2['GDMP']-avg)/std),1)
GDMP_grid2.loc[:,'zscore'] = round(((GDMP_grid2['GDMP']-avg)/std),1)
GDMP_grid2

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,GridNum10km,AreaHa,year,GDMP,zscore
19,9731949216137216,10000.0,2014-11-01,757097776.0,0.3
302908,9731949216137216,10000.0,2015-11-01,776060028.0,0.9
605797,9731949216137216,10000.0,2016-11-01,744887192.0,-0.1
908686,9731949216137216,10000.0,2017-11-01,798881914.0,1.6
1211575,9731949216137216,10000.0,2018-11-01,719134175.0,-1.0
1514464,9731949216137216,10000.0,2019-11-01,762323388.0,0.4
1817353,9731949216137216,10000.0,2020-11-01,727224963.0,-0.7
2120242,9731949216137216,10000.0,2021-11-01,706710740.0,-1.4


In [46]:
GDMP_grid2=df_transformed_AGBflux[df_transformed_AGBflux['GridNum10km']== 9731949216137216]
avg = GDMP_grid2['GDMP'].mean()
std = GDMP_grid2['GDMP'].std()
GDMP_grid2['zscore'] = round(((GDMP_grid2['GDMP']-avg)/std),1)
GDMP_grid2

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,GridNum10km,AreaHa,year,GDMP,zscore
19,9731949216137216,10000.0,2014-11-01,757097776.0,0.3
302908,9731949216137216,10000.0,2015-11-01,776060028.0,0.9
605797,9731949216137216,10000.0,2016-11-01,744887192.0,-0.1
908686,9731949216137216,10000.0,2017-11-01,798881914.0,1.6
1211575,9731949216137216,10000.0,2018-11-01,719134175.0,-1.0
1514464,9731949216137216,10000.0,2019-11-01,762323388.0,0.4
1817353,9731949216137216,10000.0,2020-11-01,727224963.0,-0.7
2120242,9731949216137216,10000.0,2021-11-01,706710740.0,-1.4


In [48]:
GDMP_grid3=df_transformed_AGBflux[df_transformed_AGBflux['GridNum10km']== 27603260890152960]
avg = GDMP_grid3['GDMP'].mean()
std = GDMP_grid3['GDMP'].std()
GDMP_grid3['zscore'] = round(((GDMP_grid3['GDMP']-avg)/std),1)
GDMP_grid3

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,GridNum10km,AreaHa,year,GDMP,zscore
18,27603260890152960,10000.0,2014-11-01,290296014.0,-0.9
302907,27603260890152960,10000.0,2015-11-01,293391800.0,-0.8
605796,27603260890152960,10000.0,2016-11-01,302799413.0,-0.4
908685,27603260890152960,10000.0,2017-11-01,295268642.0,-0.7
1211574,27603260890152960,10000.0,2018-11-01,347119917.0,1.5
1514463,27603260890152960,10000.0,2019-11-01,342778664.0,1.3
1817352,27603260890152960,10000.0,2020-11-01,325772260.0,0.6
2120241,27603260890152960,10000.0,2021-11-01,293333499.0,-0.8


In [41]:
avg = GDMP_grid['GDMP'].mean()
std = GDMP_grid['GDMP'].std()
GDMP_grid['zscore'] = round(((GDMP_grid['GDMP']-avg)/std),1)
GDMP_grid

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,GridNum10km,AreaHa,year,GDMP,zscore
114716,18446609658413056,10000.0,2014-11-01,567420786.0,-0.6
417605,18446609658413056,10000.0,2015-11-01,554946035.0,-1.0
720494,18446609658413056,10000.0,2016-11-01,591739566.0,0.2
1023383,18446609658413056,10000.0,2017-11-01,556189307.0,-1.0
1326272,18446609658413056,10000.0,2018-11-01,626023208.0,1.3
1629161,18446609658413056,10000.0,2019-11-01,597534185.0,0.4
1932050,18446609658413056,10000.0,2020-11-01,634120631.0,1.5
2234939,18446609658413056,10000.0,2021-11-01,565236683.0,-0.7


In [51]:
from  scipy.stats import zscore

In [57]:
z = np.around(zscore(GDMP_grid1['GDMP']),1)
z

array([-0.7, -1.1,  0.2, -1.1,  1.4,  0.4,  1.6, -0.7])

In [56]:
z = np.round_(zscore(GDMP_grid1['GDMP']),1)
z

array([-0.7, -1.1,  0.2, -1.1,  1.4,  0.4,  1.6, -0.7])

In [60]:
GDMP_grid=df_transformed_AGBflux[(df_transformed_AGBflux['GridNum10km']== 18446609658413056) | (df_transformed_AGBflux['GridNum10km']== 9731949216137216) | (df_transformed_AGBflux['GridNum10km']== 27603260890152960)]
GDMP_grid

,GridNum10km,AreaHa,year,GDMP
18,27603260890152960,10000.0,2014-11-01,290296014.0
19,9731949216137216,10000.0,2014-11-01,757097776.0
114716,18446609658413056,10000.0,2014-11-01,567420786.0
302907,27603260890152960,10000.0,2015-11-01,293391800.0
302908,9731949216137216,10000.0,2015-11-01,776060028.0
417605,18446609658413056,10000.0,2015-11-01,554946035.0
605796,27603260890152960,10000.0,2016-11-01,302799413.0
605797,9731949216137216,10000.0,2016-11-01,744887192.0
720494,18446609658413056,10000.0,2016-11-01,591739566.0
908685,27603260890152960,10000.0,2017-11-01,295268642.0


In [61]:
def z_score(x):
   z = np.around(zscore(x),1)
   return z

#df['Z_SCORE'] = df.groupby('GROUP')['VALUE'].transform(z_score)
GDMP_grid['zscore'] = GDMP_grid.groupby('GridNum10km')['GDMP'].transform(z_score)

print (GDMP_grid)

               GridNum10km   AreaHa       year         GDMP  zscore
18       27603260890152960  10000.0 2014-11-01  290296014.0    -1.0
19        9731949216137216  10000.0 2014-11-01  757097776.0     0.3
114716   18446609658413056  10000.0 2014-11-01  567420786.0    -0.7
302907   27603260890152960  10000.0 2015-11-01  293391800.0    -0.8
302908    9731949216137216  10000.0 2015-11-01  776060028.0     0.9
417605   18446609658413056  10000.0 2015-11-01  554946035.0    -1.1
605796   27603260890152960  10000.0 2016-11-01  302799413.0    -0.4
605797    9731949216137216  10000.0 2016-11-01  744887192.0    -0.1
720494   18446609658413056  10000.0 2016-11-01  591739566.0     0.2
908685   27603260890152960  10000.0 2017-11-01  295268642.0    -0.7
908686    9731949216137216  10000.0 2017-11-01  798881914.0     1.7
1023383  18446609658413056  10000.0 2017-11-01  556189307.0    -1.1
1211574  27603260890152960  10000.0 2018-11-01  347119917.0     1.6
1211575   9731949216137216  10000.0 2018-11-01  

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [62]:
# create a list of our conditions
conditions = [
    (GDMP_grid['zscore'] <= -1),
    (GDMP_grid['zscore'] > -1) & (GDMP_grid['zscore'] <= 1),
    (GDMP_grid['zscore'] > 1)
    ]
     
# create a list of the values we want to assign for each condition
values = ['-1', '0', '+1']

# create a new column and use np.select to assign values to it using our lists as arguments
GDMP_grid['ANOMALIES'] = np.select(conditions, values)

# display updated DataFrame
GDMP_grid.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,GridNum10km,AreaHa,year,GDMP,zscore,ANOMALIES
18,27603260890152960,10000.0,2014-11-01,290296014.0,-1.0,-1
19,9731949216137216,10000.0,2014-11-01,757097776.0,0.3,0
114716,18446609658413056,10000.0,2014-11-01,567420786.0,-0.7,0
302907,27603260890152960,10000.0,2015-11-01,293391800.0,-0.8,0
302908,9731949216137216,10000.0,2015-11-01,776060028.0,0.9,0


In [63]:
GDMP_grid

,GridNum10km,AreaHa,year,GDMP,zscore,ANOMALIES
18,27603260890152960,10000.0,2014-11-01,290296014.0,-1.0,-1
19,9731949216137216,10000.0,2014-11-01,757097776.0,0.3,0
114716,18446609658413056,10000.0,2014-11-01,567420786.0,-0.7,0
302907,27603260890152960,10000.0,2015-11-01,293391800.0,-0.8,0
302908,9731949216137216,10000.0,2015-11-01,776060028.0,0.9,0
417605,18446609658413056,10000.0,2015-11-01,554946035.0,-1.1,-1
605796,27603260890152960,10000.0,2016-11-01,302799413.0,-0.4,0
605797,9731949216137216,10000.0,2016-11-01,744887192.0,-0.1,0
720494,18446609658413056,10000.0,2016-11-01,591739566.0,0.2,0
908685,27603260890152960,10000.0,2017-11-01,295268642.0,-0.7,0
